# ChatBot RAG com documentos PDF
---
Este notebook cria uma classe chatbot com memoria para interagir.

### IMPORTS

In [1]:
import os
import time
from dotenv import load_dotenv
from typing import Dict, Any, List

## Groq
from langchain_groq import ChatGroq

## core
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

### CONFIGURAÇÃO DA API

In [3]:
load_dotenv()

GROQ_API_KEY = os.environ["GROQ_API_KEY"]

if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY não foi configurada!")

print("API Key configurada!")

API Key configurada!


### CONFIGURAÇÕES GLOBAIS

In [4]:
CONFIGS = {
    "model":"llama-3.1-8b-instant",
    "temperature": 0.1,
    "max_tokens": 2048,
    "max_messages":10
}

print(f"""
CONFIGURAÇÕES
    - Modelo LLM              - {CONFIGS['model']}
    - Temperatura             - {CONFIGS['temperature']}
    - Max tokens              - {CONFIGS['max_tokens']}
    - Max mensagens histórico - {CONFIGS['max_messages']}
""")


CONFIGURAÇÕES
    - Modelo LLM              - llama-3.1-8b-instant
    - Temperatura             - 0.1
    - Max tokens              - 2048
    - Max mensagens histórico - 10



### FUNÇÕES AUXILIARES

In [5]:
# check conexão
def test_groq_connection() -> bool:
    """Testa a conexão com a API Groq."""
    try:
        llm = ChatGroq(
            model=CONFIGS['model'],
            temperature=0,
            api_key=GROQ_API_KEY
        )
        res = llm.invoke('Responda apenas: OK')
        if res.content.strip().upper() == 'OK':
            return True
        else:
            return False
    except Exception as e:
        print(f'Erro: {str(e)}')
        return False

### CLASSE PRINCIPAL

In [14]:
class ChatBot:
    """
    ChatBot para conversar através da API Groq
    """

    def __init__(self, configs:Dict[str,Any]=CONFIGS):
        """Inicializa o chatbot"""
        self.configs = configs
        self.llm = None
        self.history:List[Any] = []

        print('=== INICIALIZANDO CHATBOT ===')
        self._initialize_llm()
        self._initialize_system_prompt()

    def _initialize_llm(self):
        """Inicializa o modelo de linguagem Groq"""
        self.llm = ChatGroq(
            model=self.configs['model'],
            temperature=self.configs['temperature'],
            max_tokens=self.configs['max_tokens'],
            api_key=GROQ_API_KEY
        )
        print(f'Modelo {self.configs["model"]} carregado!')

    def _initialize_system_prompt(self):
        """Mensagem Fixa"""
        system_message = SystemMessage(
            content='Você é um assistente prestativo que responde perguntas objetivamente.'
        )
        self.history.append(system_message)

    def chat(self, user_input, verbose=True):
        """Processa um pergunta do usuário e retorna a resposta"""
        
        start_time = time.time()
        
        self.history.append(HumanMessage(content=user_input))
        res = self.llm.invoke(self.history)
        self.history.append(AIMessage(content=res.content))

        elapsed_time = time.time() - start_time

        if len(self.history)>self.configs['max_messages']:
            self.history = self.history[-self.configs['max_messages']:]
        
        if verbose:
            print(f'USER: {user_input}\n')
            print(f'BOT: {res.content}\nTime: {elapsed_time:2f}s')
            print(f'\n\nHistorico: {len(self.history)}')
        else:
            print(res.content)
    
    def clear_history(self):
        """Limpa o histórico de conversação"""
        self.history = []
        print('Histórico limpo')

### INICIALIZAR CHATBOT

In [15]:
if test_groq_connection():
    bot = ChatBot()

=== INICIALIZANDO CHATBOT ===
Modelo llama-3.1-8b-instant carregado!


### FAZER PERGUNTAS

In [16]:
bot.chat('Qual a capital da França')

USER: Qual a capital da França

BOT: A capital da França é Paris.
Time: 0.210154s


Historico: 3


In [17]:
bot.chat('Resuma econometria em uma frase.')

USER: Resuma econometria em uma frase.

BOT: A econometria é a aplicação de métodos estatísticos e matemáticos para analisar e prever comportamentos econômicos, ajudando a tomar decisões informadas em áreas como política econômica, finanças e planejamento.
Time: 0.275094s


Historico: 5


In [18]:
bot.chat('Explique machine learning em forma de poema',verbose=False)

Em reinos de dados, onde informações se escondem,
Um novo rei surge, com poderes mágicos adornados.
Seu nome é Machine Learning, uma arte sublime,
Que ajuda a prever, a classificar e a encontrar o tempo.

Com algoritmos que aprendem, e ajustam-se ao passo,
Ele analisa os dados, e descobre o que é preciso.
A partir de exemplos, ele constrói regras e leis,
Para prever o futuro, e evitar surpresas ruins.

Ele é treinado com dados, que são alimentados e purificados,
E a partir deles, ele aprende a reconhecer padrões e nuances.
Com cada nova entrada, ele ajusta seu modelo,
E se torna mais preciso, em sua previsão e classificação.

Mas não é apenas prever, que é o seu poder,
Ele também pode ajudar a encontrar soluções, e a melhorar.
Com análises avançadas, ele pode identificar problemas,
E sugerir soluções, que sejam eficazes e eficientes.

Então, Machine Learning é um poderoso aliado,
Que ajuda a tomar decisões, com base em dados e fatos.
Ele é uma ferramenta valiosa, que pode ser usada,
Pa

In [19]:
bot.chat('resuma em uma frase: o que sao demonstrações financeiras na contabilidade?')

USER: resuma em uma frase: o que sao demonstrações financeiras na contabilidade?

BOT: As demonstrações financeiras são relatórios que apresentam a situação financeira e o desempenho de uma empresa, incluindo o balanço patrimonial, o resultado do exercício e o fluxo de caixa, com o objetivo de fornecer informações precisas e confiáveis sobre a empresa.
Time: 0.386994s


Historico: 9


In [20]:
bot.chat('qual foi minha primeira pergunta?')

USER: qual foi minha primeira pergunta?

BOT: Sua primeira pergunta foi sobre a capital da França.
Time: 0.236057s


Historico: 10


In [21]:
bot.clear_history()

Histórico limpo


In [22]:
bot.chat('qual foi minha primeira pergunta?')

USER: qual foi minha primeira pergunta?

BOT: Sua primeira pergunta foi: "qual foi minha primeira pergunta?"
Time: 0.219652s


Historico: 2
